<a href="https://colab.research.google.com/github/softwarerbfl/TABA_AI_Project/blob/main/AlexNetModeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 전처리

In [2]:
import numpy as np
import pandas as pd
import os
from PIL import Image
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
import cv2
import base64
import zipfile
import imageio
import torch
import h5py
from google.colab import drive
import torchvision
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torch.autograd import Variable
from torch import optim
import torch.nn as nn
import torch.nn.functional as F
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [108]:
#저작권 이미지 경로
cpImage_path="/content/drive/MyDrive/TABA_AI/Image/copyrighted/copyrightedImage"
#비저작권 이미지 경로
noncpImage_path="/content/gdrive/My Drive/TABA_AI/Image/noncopyrighted/ILSVRC2010_val_"

**평가지표 측정 함수**

In [109]:
def evaluation_index(predict, real):
  TP=0.0
  TN=0.0
  FP=0.0
  FN=0.0
  for i in range(len(predict)):
    for j in range(len(predict[i])):
      if predict[i][j]==real[i][j]:
       if predict[i][j]==1:
         TN+=1
       else:
         TP+=1
      else:
       if predict[i][j]==0:
        FP+=1
       else:
        FN+=1

    
  accuracy=(TP+TN)/(TP+TN+FP+FN)
  recall=TP/(TP+FN)
  precision=TP/(TP+FP)
  F1=(2*precision*recall)/(precision+recall)
  print("accuracy:",accuracy)
  print("recall:",recall)
  print("precision:",precision)
  print("F1:",F1)

**DataLoader로 이미지 데이터 가져오기**

In [110]:
BATCH_SIZE=16

In [111]:
from torchvision import datasets, transforms, models

train_transforms = transforms.Compose([
                                       transforms.CenterCrop(256),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([
                                      transforms.CenterCrop(256),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

# train_data = datasets.ImageFolder(DATASET_PATH + '/train', transform=train_transforms)
# test_data = datasets.ImageFolder(DATASET_PATH + '/test', transform=test_transforms)

# train_iter=torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=1)
# test_iter=torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=1)

print("Preparing dataset done")

Preparing dataset done


In [112]:
from torchvision.datasets import ImageFolder
train_dataset=ImageFolder("/content/drive/MyDrive/TABA_AI/Image/Folder/train",transform=train_transforms)
test_dataset=ImageFolder("/content/drive/MyDrive/TABA_AI/Image/Folder/test",transform=test_transforms)

In [113]:
train_iter=DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=1)
test_iter=DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=1)

# AlexNet

**모델 선언**

In [114]:
from torch import nn

class AlexNet(nn.Module):
  def __init__(self):
    super().__init__()
    self.CNN=nn.Sequential(
        # MaxPooling층: 데이터의 중요한 요소들만 요약하여 추출
        nn.Conv2d(in_channels=3, out_channels=96, kernel_size=11, stride=4, padding=0),
        nn.ReLU(inplace=True),
        nn.LocalResponseNorm(size=5, k=2),
        nn.MaxPool2d(kernel_size=3, stride=2),

        # MaxPooling층: 데이터의 중요한 요소들만 요약하여 추출
        nn.Conv2d(in_channels=96, out_channels=256, kernel_size=5, stride=1, padding=2),
        nn.ReLU(inplace = True),
        nn.LocalResponseNorm(size=5, k=2),
        nn.MaxPool2d(kernel_size=3, stride=2),

        # 중간다리 역할 
        nn.Conv2d(in_channels=256, out_channels=384, kernel_size=3, stride=1, padding=1),
        nn.ReLU(inplace = True),
        nn.LocalResponseNorm(size=5, k=2),

        # 중간다리 역할 
        nn.Conv2d(in_channels=384, out_channels=384, kernel_size=3, stride=1, padding=1),
        nn.ReLU(inplace = True),
        nn.LocalResponseNorm(size=5, k=2),

        # 중간다리 역할 
        nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, stride=1, padding=1),
        nn.ReLU(inplace = True),
        nn.LocalResponseNorm(size=5, k=2),

        # MaxPooling층 : Fully Connected 층 2개로 구성 
        nn.MaxPool2d(kernel_size=3, stride=2),
    )
    self.FC = nn.Sequential(
        nn.Linear(256*6*6, 4096),
        nn.ReLU(True),
        nn.Dropout(0.5),

        nn.Linear(4096, 4096),
        nn.ReLU(True),
        nn.Dropout(0.5),

        nn.Linear(4096, 2)
    )
  def forward(self, inp):
    cnn_res = self.CNN(inp)
    flatten = torch.flatten(cnn_res, 1)
    fc_res = self.FC(flatten)
    return fc_res

In [115]:
model = AlexNet()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)

AlexNet(
  (CNN): Sequential(
    (0): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4))
    (1): ReLU(inplace=True)
    (2): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (5): ReLU(inplace=True)
    (6): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2)
    (7): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2)
    (11): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2)
    (14): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): LocalResponseNorm(5, alpha=0.0001, beta

**train(훈련) 함수 선언**

In [116]:
def test_eval(model, data_iter, batch_size):
  return_y=[]
  return_predict=[]
  with torch.no_grad():
    test_loss = 0
    total = 0
    correct = 0
    model.eval()
    for batch_img, batch_lab in data_iter:
      X = batch_img.to(device)
      Y = batch_lab.to(device)
      y_pred = model(X)
      _, predicted = torch.max(y_pred.data, 1)
      correct += (predicted == Y).sum().item()
      total += batch_img.size(0)
      return_y.append(Y.tolist())
      return_predict.append(predicted.tolist())

    
    val_acc = (100*correct / total)
    model.train()
  # print("real:",Y)
  # print("predict:",predicted)
  return val_acc,return_y,return_predict #정확도, 실제값, 예측값 반환

In [117]:
from torch import optim
lr=0.0001
optimizer=optim.NAdam(model.parameters(), lr=lr)
criterion=nn.CrossEntropyLoss() #손실함수
EPOCHS=30
avg_acc=[]
print("Batch Size:",BATCH_SIZE, ",Learning Rate:",lr)
## Training Phase
print_every = 1
print("Start training !")

# Training loop
for epoch in range(EPOCHS):
  loss_val_sum = 0
  for batch_img, batch_lab in train_iter:

    X = batch_img.to(device)
    Y = batch_lab.to(device)

    # inference & Calculate loss
    y_pred = model.forward(X)
    loss = criterion(y_pred, Y)

    # print(y_pred)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    loss_val_sum += loss

  if ((epoch%print_every)==0) or (epoch==(EPOCHS-1)):
    #accr_val = M.test(x_test, y_test, batch_size)
    loss_val_avg = loss_val_sum / len(train_iter)
    accr_val,_,_ = test_eval(model, train_iter, BATCH_SIZE)
    print("epoch:",epoch+1,"/",EPOCHS," costs:",loss_val_avg.item(), " test_accuracy:",accr_val)
    avg_acc.append(accr_val)
print("Training Done! ")
print("average accuracy:",sum(avg_acc)/len(avg_acc))

Batch Size: 16 ,Learning Rate: 0.0001
Start training !
epoch: 1 / 30  costs: 0.6313969492912292  test_accuracy: 69.8125
epoch: 2 / 30  costs: 0.5808980464935303  test_accuracy: 72.825
epoch: 3 / 30  costs: 0.5561648011207581  test_accuracy: 71.575
epoch: 4 / 30  costs: 0.533268392086029  test_accuracy: 74.4125
epoch: 5 / 30  costs: 0.5147956609725952  test_accuracy: 72.4625
epoch: 6 / 30  costs: 0.4860383868217468  test_accuracy: 75.225
epoch: 7 / 30  costs: 0.4638483226299286  test_accuracy: 80.275
epoch: 8 / 30  costs: 0.431165486574173  test_accuracy: 83.3875
epoch: 9 / 30  costs: 0.4042447507381439  test_accuracy: 85.5375
epoch: 10 / 30  costs: 0.36717864871025085  test_accuracy: 86.6125
epoch: 11 / 30  costs: 0.31184905767440796  test_accuracy: 90.1125
epoch: 12 / 30  costs: 0.2591738700866699  test_accuracy: 93.125
epoch: 13 / 30  costs: 0.2010347545146942  test_accuracy: 92.7125
epoch: 14 / 30  costs: 0.15873700380325317  test_accuracy: 96.15
epoch: 15 / 30  costs: 0.11901718378

**테스트**

In [118]:
model.eval()
accr_val,Y,predicted = test_eval(model, test_iter, BATCH_SIZE)
evaluation_index(predicted,Y)

accuracy: 0.7385
recall: 0.774
precision: 0.7226890756302521
F1: 0.7474649927571222
